# 헤더

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install music21

In [3]:
!pip install midi2audio

In [4]:
!pip install midiutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for midiutil: filename=MIDIUtil-1.2.1-py3-none-any.whl size=54571 sha256=7825d24df2f28c1d42d9ed905cc173b4ef66eeaee8e7dd19abea6b8bedd5a726
  Stored in directory: /root/.cache/pip/wheels/af/43/4a/00b5e4f2fe5e2cd6e92b461995a3a97a2cebb30ab5783501b0
Successfully built midiutil


In [5]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
from music21 import converter, note, stream, environment
from midi2audio import FluidSynth
from midiutil import MIDIFile

#계이름 주파수 도메인 정의

In [6]:
# hertz2keys 데이터프레임 정의
hertz2keys = pd.DataFrame({
    'A': [27.50, 55.00, 110.00, 220.00, 440.00, 880.00, 1760.00, 3520.00, 7040.00],
    'B': [30.87, 61.74, 123.47, 246.94, 493.88, 987.77, 1975.53, 3951.07, 7902.13],
    'Bb': [29.14, 58.27, 116.54, 233.08, 466.16, 932.33, 1864.66, 3729.31, 7458.62],
    'C': [16.35, 32.70, 65.41, 130.81, 261.63, 523.25, 1046.50, 2093.00, 4186.01],
    'C#': [17.32, 34.65, 69.30, 138.59, 277.18, 554.37, 1108.73, 2217.46, 4434.92],
    'D': [18.35, 36.71, 73.42, 146.83, 293.66, 587.33, 1174.66, 2349.32, 4698.64],
    'D#': [19.45, 38.89, 77.78, 155.56, 311.13, 622.25, 1244.51, 2489.02, 4978.03],
    'E': [20.60, 41.20, 82.41, 164.81, 329.63, 659.26, 1318.51, 2637.02, 5274.04],
    'F': [21.83, 43.65, 87.31, 174.61, 349.23, 698.46, 1396.91, 2793.83, 5587.65],
    'F#': [23.12, 46.25, 92.50, 185.00, 369.99, 739.99, 1479.98, 2959.96, 5919.91],
    'G': [24.50, 49.00, 98.00, 196.00, 392.00, 783.99, 1567.98, 3135.96, 6271.93],
    'G#': [25.96, 51.91, 103.83, 207.65, 415.30, 830.61, 1661.22, 3322.44, 6644.88]
}, index=['0', '1', '2', '3', '4', '5', '6', '7', '8'])

#오디오 불러오기
slice_duration = 0.01~0.2 추천

In [7]:
# 오디오 파일 경로
audio_path = '/content/drive/MyDrive/tfile.mp3'

# 오디오 불러오기
y, sr = librosa.load(audio_path, sr=None)

# 슬라이싱 간격 설정 (0.01초)
slice_duration = 0.05
slice_samples = int(slice_duration * sr)

# FFT를 위한 윈도우 크기 설정
n_fft = 2048

# 결과를 저장할 데이터프레임 생성
result_df = pd.DataFrame(columns=['start_time', 'end_time', 'key'])

In [8]:
for i in range(0, len(y), slice_samples):
    # 슬라이싱된 오디오 데이터
    audio_slice = y[i:i+slice_samples]

    # FFT 적용
    spectrum = np.abs(librosa.stft(audio_slice, n_fft=n_fft))

    # 슬라이싱된 부분에서 주파수가 가장 큰 부분 추출
    max_freq_index = np.argmax(np.sum(spectrum, axis=1))

    # 주파수가 0이 아니라면 처리
    if max_freq_index > 0:
        # 해당 주파수에 대응되는 계이름 찾기
        hz = max_freq_index * sr / n_fft

        # 주파수에 대응되는 키 찾기
        key = None
        min_diff = float('inf')
        for column in hertz2keys.columns:
            diff = np.abs(hz - hertz2keys[column].values)
            min_index = np.argmin(diff)

            if diff[min_index] < min_diff:
                min_diff = diff[min_index]
                key = f'{column}{hertz2keys.index[min_index]}'

        # 결과 데이터프레임에 추가
        result_df = pd.concat([result_df, pd.DataFrame({
            'start_time': [i / sr],
            'end_time': [(i + slice_samples) / sr],
            'key': [key] if key else ['Unknown']
        })], ignore_index=True)
    else:
        # 주파수가 0인 경우 'Unknown'으로 처리
        result_df = pd.concat([result_df, pd.DataFrame({
            'start_time': [i / sr],
            'end_time': [(i + slice_samples) / sr],
            'key': ['Unknown']
        })], ignore_index=True)


/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=477
  warnings.warn(


In [9]:
result_df

,start_time,end_time,key
0,0.00,0.05,Unknown
1,0.05,0.10,Unknown
2,0.10,0.15,Unknown
3,0.15,0.20,Unknown
4,0.20,0.25,Unknown
...,...,...,...
443,22.15,22.20,Unknown
444,22.20,22.25,Unknown
445,22.25,22.30,Unknown
446,22.30,22.35,Unknown


#자른 음표 다시 합치기

In [10]:
# 연속으로 나오는 같은 음표를 합치기 위한 임계값 설정 (예: 0.1초)
threshold = 0.25

# 새로운 데이터프레임 생성
combined_df = pd.DataFrame(columns=['start_time', 'end_time', 'key'])

# 초기 값 설정
current_start_time = result_df.loc[0, 'start_time']
current_end_time = result_df.loc[0, 'end_time']
current_key = result_df.loc[0, 'key']

# 결과 데이터프레임을 순회하면서 연속 음표 합치기
for index, row in result_df.iloc[1:].iterrows():
    if row['start_time'] - current_end_time <= threshold and row['key'] == current_key:
        # 현재 음표를 연장
        current_end_time = row['end_time']
    else:
        # 새로운 음표 시작
        combined_df = pd.concat([combined_df, pd.DataFrame({
            'start_time': [current_start_time],
            'end_time': [current_end_time],
            'key': [current_key]
        })], ignore_index=True)

        # 초기 값 갱신
        current_start_time = row['start_time']
        current_end_time = row['end_time']
        current_key = row['key']

# 마지막 음표 추가
combined_df = pd.concat([combined_df, pd.DataFrame({
    'start_time': [current_start_time],
    'end_time': [current_end_time],
    'key': [current_key]
})], ignore_index=True)

In [11]:
combined_df

,start_time,end_time,key
0,0.00,1.10,Unknown
1,1.10,1.15,G4
2,1.15,1.55,D4
3,1.55,1.80,G5
4,1.80,1.90,G4
5,1.90,2.15,Bb5
6,2.15,2.25,Bb4
7,2.25,3.40,D5
8,3.40,3.75,C5
9,3.75,4.05,Bb5


#미디 파일 설정

In [13]:
from music21 import stream, note, instrument

# MIDI 파일 생성을 위한 설정
midi_stream = stream.Score()

# 스트림에 악기 추가 (예: 'Violin' 악기)
midi_stream.append(instrument.Violin())

# 처음에 레스트 추가
midi_stream.append(note.Rest(quarterLength=1.0))

# 키에 따라 MIDI 이벤트 추가
for index, row in combined_df.iterrows():
    key = row['key']

    # 'Unknown' 값이 아닌 경우에만 MIDI 노트 이벤트 추가
    if key != 'Unknown':
        start_time = row['start_time']
        end_time = row['end_time']

        # MIDI 노트 이벤트 추가
        n = note.Note(key, quarterLength=end_time - start_time)

        # 노트에 악기 설정
        n.instrument = instrument.Violin()  # 다른 악기로 변경 가능

        midi_stream.append(n)

# MIDI 파일 저장
midi_stream.write('midi', fp='/content/drive/MyDrive/AI_composer/Outputs/outputs.mid')


'/content/drive/MyDrive/AI_composer/Outputs/outputs.mid'